# Empirical Analysis of Market Capitalization Weighted Index (MWI) vs. Equal Weighted Index (EWI) Performance for S&P 500

This notebook presents an empirical analysis of the performance of Market Capitalization Weighted Indexing (MWI) and Equal Weighted Indexing (EWI) strategies applied to a subset of the S&P 500. The objective is to explore why the EWI strategy often outperforms the MWI strategy.

For the sake of stability and simplicity in this analysis, we are working with the 411 companies that have continuously been part of the S&P 500 from January 2006 to August 2023. This eliminates any effects due to companies entering or leaving the index during this period.

We begin by fetching the necessary data for these companies using open-source Python package like `yfinance` and web scraping techniques. All the data used in this analysis is freely available and no proprietary or paid sources were used.

Next, we build MWI and EWI portfolios from this data, calculating returns and other relevant metrics.Please note that we do not consider transaction fees in these custum portfolios to avoid additional complexity. Furthermore, for the rebalancing of both portfolios, we perform it after the close of the third Friday in March, June, September, and December. 

Through exploratory data analysis, we delve into the distribution characteristics of individual stock returns, focusing on their skewness as a potential explanatory factor for the observed performance difference. We also visualize various aspects of our portfolios to better understand their composition and performance over time.

Let's begin

## Step 1: Fetching the Tickers of S&P 500 Constituents

The function `get_sp500_tickers()` fetches the ticker symbols of all companies that are currently constituents of the S&P 500 index.

This is done by web scraping the Wikipedia page that lists the S&P 500 constituents. We are using the pandas `read_html` function which conveniently fetches tabular data from an HTML page and returns it in the form of a DataFrame. After fetching the data, we just extract the ticker symbols and return them as a list. 



In [75]:
import pandas as pd 

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url, header=0)
    df = table[0]
    return list(df.Symbol)

After running this function, we will have a list of ticker symbols for all current S&P 500 constituents stored in the `tickers` variable.


In [4]:
tickers = get_sp500_tickers()

In [10]:
sorted(tickers)[:10]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN', 'ADBE']

## Step 1 Fetching Historical Market Capitalization Data 

To construct our **Market Capitalization Weighted Index (MWI)**, we need to collect historical daily market capitalization data for each constituent of the S&P 500. Sourcing this type of data can often be challenging due to access restrictions and cost constraints. After thorough exploration, I found a freely available API endpoint on StockAnalysis.com that supplies this precise information.

This endpoint was discovered by examining the Network tab in the browser's Developer Tools while interacting with the `stockanalysis.com` website. This allowed us to observe the API calls made by the web application.

The API endpoint we identified is `https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price`, where `{symbol}` the ticker symbol of a stock we're interested in. This API call returns a JSON object with a `status` field, indicating the success or failure of the request, and a `data` field containing the actual market cap data. The `data` field is an array of arrays, where each inner array consists of a Unix timestamp (representing the date) and the market cap for the specified stock on that date.

To effectively gather data for all the stocks in our list, I implemented a concurrent fetching system `concurrent.futures` module. This system allows us to send multiple API requests simultaneously, significantly reducing the total time required to gather all our necessary data.



In the following code, we will pull historical market capitalization data for all currently listed stocks in the S&P 500 index :




In [11]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_fixed

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def get_market_cap(symbol):
    url = f"https://stockanalysis.com/api/symbol/s/{symbol}/marketcap?t=price"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame({
        'Date': [pd.to_datetime(x[0], unit='ms') for x in data['data']],
        f'Market Cap {symbol}': [x[1] for x in data['data']]
    })

def get_market_caps(tickers):
    first_ticker = tickers.pop(0)
    master_df = get_market_cap(first_ticker)

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_ticker = {executor.submit(get_market_cap, ticker): ticker for ticker in tickers}
        for future in as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                df = future.result()
                master_df = master_df.merge(df, on='Date', how='outer')
            except Exception as e:
                print(f"Error occurred while getting market cap data for {ticker}: {e}")

    return master_df

In [12]:
data = get_market_caps(tickers)

In [42]:
data.set_index('Date', inplace=True)

In [43]:
data.head()

,Market Cap MMM,Market Cap ABBV,Market Cap AOS,Market Cap ABT,Market Cap ACN,Market Cap ATVI,Market Cap ADM,Market Cap ADBE,Market Cap ADP,Market Cap AAP,...,Market Cap WTW,Market Cap GWW,Market Cap WYNN,Market Cap XEL,Market Cap XYL,Market Cap ZBRA,Market Cap ZBH,Market Cap YUM,Market Cap ZION,Market Cap ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1998-12-01,3.266150e+10,NaN,610000000.0,7.266370e+10,NaN,NaN,NaN,3.018500e+09,2.330110e+10,NaN,...,NaN,4.106500e+09,NaN,NaN,NaN,1.046800e+09,NaN,7.022000e+09,4.022400e+09,NaN
1998-12-02,3.248490e+10,NaN,612800000.0,7.418130e+10,NaN,NaN,NaN,2.957000e+09,2.398090e+10,NaN,...,NaN,4.095200e+09,NaN,NaN,NaN,1.044900e+09,NaN,6.936400e+09,4.100600e+09,NaN
1998-12-03,3.188290e+10,NaN,611400000.0,7.266370e+10,NaN,NaN,NaN,3.018500e+09,2.284800e+10,NaN,...,NaN,4.112200e+09,NaN,NaN,NaN,1.039000e+09,NaN,7.098400e+09,4.171000e+09,NaN
1998-12-04,3.220800e+10,NaN,612800000.0,7.550170e+10,NaN,NaN,NaN,2.965400e+09,2.339480e+10,NaN,...,NaN,4.207200e+09,NaN,NaN,NaN,1.039000e+09,NaN,7.537400e+09,4.257000e+09,NaN
1998-12-07,3.092770e+10,NaN,598400000.0,7.550170e+10,NaN,NaN,NaN,3.064200e+09,2.358210e+10,NaN,...,NaN,4.142600e+09,NaN,NaN,NaN,1.062400e+09,NaN,7.547800e+09,4.261700e+09,NaN


  ## Step 2 Data Cleaning
  <span style ="color:blue"> *Let's analyze the number of non-NaN values for each year in our dataset. 
  This will tells us the number of companies in our dataset that existed in each year.*</span>


In [48]:
import numpy as np
non_zero_counts = data.groupby(data.index.year).count().replace(0, np.nan).apply(lambda x: x.notnull().sum(), axis=1)
non_zero_counts

Date
1983      2
1984      2
1985      2
1986      2
1987      3
1988      3
1989      3
1990      3
1991      3
1992      3
1993      3
1994      3
1995      3
1996      3
1997      3
1998    334
1999    349
2000    363
2001    384
2002    388
2003    393
2004    405
2005    412
2006    418
2007    427
2008    431
2009    435
2010    444
2011    450
2012    457
2013    468
2014    476
2015    483
2016    485
2017    489
2018    494
2019    498
2020    500
2021    501
2022    502
2023    503
dtype: int64

<span style="color:blue">*Starting from 2013 offers a comprehensive perspective because it provides over a decade of data coverage and represents more than 95% of the current S&P 500 companies.*</span>


In [57]:
data_2013 = data[data.index.year >= 2013]

df = data_2013.dropna(axis=1)

df.head()

,Market Cap MMM,Market Cap ABBV,Market Cap AOS,Market Cap ABT,Market Cap ACN,Market Cap ATVI,Market Cap ADM,Market Cap ADBE,Market Cap ADP,Market Cap AAP,...,Market Cap WHR,Market Cap WTW,Market Cap GWW,Market Cap WYNN,Market Cap XEL,Market Cap XYL,Market Cap ZBRA,Market Cap ZBH,Market Cap YUM,Market Cap ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,6.558120e+10,5.548960e+10,2.982800e+09,5.066040e+10,5.487730e+10,1.217860e+10,1.888110e+10,1.898040e+10,2.834680e+10,5.318200e+09,...,8.343600e+09,5.924800e+09,1.443610e+10,1.187570e+10,1.335100e+10,5.109300e+09,2.083100e+09,1.174610e+10,3.034350e+10,4.049900e+09
2013-01-03,6.550510e+10,5.503140e+10,2.936300e+09,5.258880e+10,5.467860e+10,1.225090e+10,1.872960e+10,1.868830e+10,2.845850e+10,5.318200e+09,...,8.322600e+09,5.985300e+09,1.439020e+10,1.200340e+10,1.331200e+10,5.092600e+09,2.085200e+09,1.191440e+10,3.053320e+10,4.055400e+09
2013-01-04,6.598950e+10,5.433780e+10,2.968300e+09,5.227270e+10,5.498060e+10,1.242890e+10,1.924330e+10,1.887640e+10,2.871000e+10,5.401100e+09,...,8.251700e+09,5.983600e+09,1.448410e+10,1.214820e+10,1.336080e+10,5.085200e+09,2.068500e+09,1.197510e+10,3.086760e+10,4.171400e+09
2013-01-07,6.607250e+10,5.444680e+10,2.935400e+09,5.269950e+10,5.474220e+10,1.244000e+10,1.844640e+10,1.878240e+10,2.859930e+10,5.382700e+09,...,8.125600e+09,5.966300e+09,1.439370e+10,1.218950e+10,1.321940e+10,5.018300e+09,2.080100e+09,1.200460e+10,3.067330e+10,4.151200e+09
2013-01-08,6.607940e+10,5.326180e+10,2.991300e+09,5.271530e+10,5.506000e+10,1.222860e+10,1.866380e+10,1.888140e+10,2.876430e+10,5.294700e+09,...,8.175400e+09,5.936800e+09,1.415330e+10,1.217890e+10,1.324370e+10,4.957000e+09,2.081600e+09,1.202200e+10,2.938560e+10,4.068300e+09


<span style="color:blue">**Note:** *There's a discrepancy in the number of companies reported for 2013 pre (468) and post data cleaning (458). This is due to Partial Data. During our initial aggregation, companies with minimal data points for 2013 were included. However, post-cleaning, companies lacking substantial data coverage for the entire year were filtered out, leading to the reduced count. This ensures our dataset only includes companies with data uniformly spread over the years.*</span>


### Step 3: Yahoo Finance Data Integration

Here's what we're doing in this step:

1. **Ticker Cleaning**: 
   <div style="margin-left: 20px">Ensuring  tickers are Yahoo Finance compatible.</div>
   
2. **Ticker Conversion**: 
   <div style="margin-left: 20px">Extracting and converting tickers from the dataset's columns for querying.</div>

3. **Date Range**: 
   <div style="margin-left: 20px">Establishing the date range using the min and max dates from the dataset.</div>

4. **Yahoo Finance Query**: 
   <div style="margin-left: 20px">Fetching price data for our tickers within the specified date range using `yfinance`.</div>

5. **DataFrame Prep**: 
   <div style="margin-left: 20px">Adjusting the column structure of our dataset for the integration with the Yahoo Finance data.</div>

6. **Data Merge**: 
   <div style="margin-left: 20px">Concatenating the Yahoo Finance data with our market data dataset.</div>





In [76]:
import yfinance as yf


tickers = [col.replace('Market Cap ', '') for col in df.columns]
tickers = [ticker.replace("BF.B", "BF-B").replace("BRK.B", "BRK-B") for ticker in tickers]
tickers_str = ' '.join(tickers)

start_date = df.index.min()
end_date = df.index.max()

yf_data = yf.download(tickers_str, start=start_date, end=end_date, group_by='ticker')

df_copy = df.copy()
df_copy.columns = pd.MultiIndex.from_product([tickers, ['Market Cap']])


merged_data = pd.concat([yf_data, df_copy], axis=1)


[*********************100%***********************]  458 of 458 completed


In [77]:
merged_data = merged_data.sort_index(axis=1)

In [78]:
merged_data.head()

A                                                 \
            Adj Close      Close       High        Low    Market Cap   
Date                                                                   
2013-01-02  27.338417  29.957083  30.164520  29.434908  1.457150e+10   
2013-01-03  27.436333  30.064379  30.143063  29.742489  1.462370e+10   
2013-01-04  27.978144  30.658083  30.722462  29.942776  1.491250e+10   
2013-01-07  27.775774  30.436338  30.615166  30.314735  1.480460e+10   
2013-01-08  27.553833  30.193132  30.600859  30.114449  1.468630e+10   

                                        AAP                        ...  \
                 Open     Volume  Adj Close      Close       High  ...   
Date                                                               ...   
2013-01-02  30.164520  8790205.0  66.418594  72.489998  73.760002  ...   
2013-01-03  30.000000  5751791.0  66.418594  72.489998  73.440002  ...   
2013-01-04  30.143063  6432897.0  67.453972  73.620003  73.639999  ...   
2013-01-07  30.472103  3589505.0  67.224915  73.370003  73.800003  ...   
2013-01-08  30.379112  3896925.0  66.125397  72.169998  73.660004  ...   

                    ZBRA                            ZION             \
              Market Cap       Open    Volume  Adj Close      Close   
Date                                                                  
2013-01-02  2.083100e+09  40.040001  213800.0  18.007875  21.990000   
2013-01-03  2.085200e+09  40.790001   99900.0  18.032444  22.020000   
2013-01-04  2.068500e+09  40.509998  146000.0  18.548361  22.650000   
2013-01-07  2.080100e+09  40.570000   90600.0  18.458277  22.540001   
2013-01-08  2.081600e+09  40.650002  112600.0  18.089771  22.090000   

                                                                      
                 High        Low    Market Cap       Open     Volume  
Date                                                                  
2013-01-02  22.000000  21.780001  4.049900e+09  21.879999  2551100.0  
2013-01-03  22.129999  21.809999  4.055400e+09  21.990000  2267900.0  
2013-01-04  22.790001  21.969999  4.171400e+09  22.020000  3577700.0  
2013-01-07  22.580000  22.260000  4.151200e+09  22.520000  2286000.0  
2013-01-08  22.530001  22.070000  4.068300e+09  22.500000  3758600.0  

[5 rows x 3206 columns]